<h2>Term Implementations</h2>

<p>Also contains recursive reduceSize() code to compute s<sub>dag</sub>.</p>
<p>Sorry everything isnt completely cleaned up yet.</p>


In [1]:
#deepcopy used to create repeated terms while keeping the original (that is possibly used elsewhere) intact.
from copy import deepcopy

#Assumtions:
#All symbols are unique.
#Integers can NOT be used as symbols, as they are used for internal variable use.

In [2]:
#Implementation of contsant. Overrides str, len, eq, and hash.
class Constant:
    registry = set()
    
    def __init__(self, s):
        self.sym = s
        self.arity = 0
        self.length = 1
        Constant.registry.add(self)
        
    def __str__(self):
        return self.sym
        
    def __len__(self):
        return self.length
    
    #def __eq__(self, other):
    #    return isinstance(other, Constant) and self.sym == other.sym
    
    def __eq__(self, other):
        return str(self) == str(other)

    def __hash__(self):
        return hash(str(self))
    
    #Tested different naming scheme. Not currently in use.
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Constant.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #Constant size can not be reduced. Higher-level structures can determine if it is a repeat.
    def reduceSize(self):
        pass
    
    def reducedSize(self):
        return self.length
    
    #Will always be a ground term. Base of higher level isGround recursion.
    def isGround(self):
        return True
    
    #No variables. Returns empty list to conform with other structures use of list concacts.
    def findVariables(self):
        return []
    
    #Is a constants. Returns object wrapped in list to conform with other structures use of list concacts.
    #Base of higher level findConstants recursion.
    def findConstants(self):
        return [self]
    
    #Is Inherently a repeated term once found. Base of higher level findRepeats recursion.
    def findRepeats(self):
        return [self]
    
    #No functions. Returns empty list to conform with other structures use of list concacts.
    def findFunctions(self):
        return []
    
    #Nothing to replace. Base of higher level replace recursion.
    def replace(self, var, rpt):
        return self
    
print "Constant structure loaded."

Constant structure loaded.


In [3]:
#Implementation of repeat. Encapsulates replaced object in case the information is needed. Length is always 0.
#Overrides str, len, eq, and hash
class Repeat:
    
    def __init__(self, original):
        self.sym = '*'
        self.origin = original
        self.length = 0
    
    #Currently shows just a star. Switch commented line for more informative printed information.
    def __str__(self):
        return self.sym + str(self.origin) + self.sym
        #return self.origin.sym
        
    def __len__(self):
        return self.length
    
    #Checks encapsulated object of equality beyond just being a repeat.
    #def __eq__(self, other):
    #    return isinstance(other, type(self.origin)) and \
    #    self.origin.sym == other.sym and self.origin.params == other.params
    
    def __eq__(self, other):
        return (str(self) == str(other)) or (str(self.origin) == str(other))
        
    def __hash__(self):
        return hash(str(self.origin))
    
    #Size already reduced. Implemented so that reduceSize still works if accidentally called multiple times.
    def reduceSize(self):
        pass
    
    def reducedSize(self):
        return self.length
    
    #Ground determined by encapsulated object. Passed on to that function.
    def isGround(self):
        return self.origin.isGround()
    
    #Variables determined by encapsulated object. Passed on to that function.
    def findVariables(self):
        return self.origin.findVariables()
    
    #Constants determined by encapsulated object. Passed on to that function.
    def findConstants(self):
        return self.origin.findConstants()
    
    #First instance of this repeat already included in search. 
    #Empty list used to conform with higher level findRepeat.
    def findRepeats(self):
        return []
    
    #Functions determined by encapsulated object. Passed on to that function.
    def findFunctions(self):
        return self.origin.findFunctions()
    
    
    #No variable to replace. Should not replace underlying object (if possible) 
    #because there is some clause that does not have this as a repeat.
    def replace(self, var, rpt):
        return self

print "Repeat structure loaded."

Repeat structure loaded.


In [4]:
#Implementaion of variable. Frequently replaced with other objects in enumerate algorithm.
#Overrides str, len, eq, and hash.
class Variable:
    
    tally = 0
    
    def __init__(self, s=0):
        if s==0:
            self.sym = self.name()
        else:
            self.sym = s
        #self.length = 1 #Use minimal possible size.
        self.length = 0
        
    def __str__(self):
        return self.sym
    
    def __len__(self):
        return self.length
    
    #def __eq__(self, other):
    #    return (isinstance(other, Variable) and self.sym == other.sym)
    
    def __eq__(self, other):
        return str(self) == str(other)
    
    def __hash__(self):
        return id(str(self))
    
    def name(self):
        Variable.tally+=1
        return str(Variable.tally)
    
    #Variable size can not be reduced. Higher-level structures can determine if it is a repeat.
    def reduceSize(self):
        pass
    
    def reducedSize(self):
        return self.length
    
    #Will never be a ground term. Base of higher level isGround recursion.
    def isGround(self):
        return False
    
    #Is a variable. Returns object wrapped in list to conform with other structures use of list concacts.
    #Base of higher level findVariables recursion.
    def findVariables(self):
        return [self]
    
    #No variables. Returns empty list to conform with other structures use of list concacts.
    def findConstants(self):
        return []
    
    #Is Inherently a repeated term once found. Base of higher level findRepeats recursion.
    def findRepeats(self):
        return []
    
    #No functions. Returns empty list to conform with other structures use of list concacts.
    def findFunctions(self):
        return []
    
    #Returns a copy of the predicate with 'var' replaced with 'rpt'.
    #Should not need to copy, because it was done by the clause.
    def replace(self, var, rpt):
        #variable = deepcopy(self)
        variable = self
        if self==var:
                return rpt
        else:
                return variable

print "Variable structure loaded."

Variable structure loaded.


In [5]:
#Implementation of function. Includes symbol (must be unique) and list of paramaters.
#Registry containts copies of all functions that may be altered in enumerate algorithm.
#Consider including arity parameter.
#Overrides str, len, eq, and hash
class Function:
    
    #Length calculated by definition: 1 + length of each parameter.
    def __init__(self, s, *args):
        self.sym = s
        self.params = list(args)
        self.arity = len(args)
        self.length = 1+sum([len(i) for i in self.params])

    def __str__(self):
        toReturn = self.sym + "("
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + ")"
        return toReturn
    
    def __len__(self):
        return 1+sum([len(i) for i in self.params])
                     
    #def __eq__(self, other):
    #    return isinstance(other, Function) and self.sym == other.sym and self.params == other.params
    
    def __eq__(self, other):
        return str(self) == str(other)
    
    def __hash__(self):
        return hash(str(self))
    
    #Previous naming scheme testing. Not in use.
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Function.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #For each parameter (0 to arity), scans forward for repeated terms. Any repeats are replaced by Repeats.
    #Then reduces size of each parameter.
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()
    
    #Scans each parameter. If any are not ground, the entire instance of the function is not ground.
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True
    
    #Scans each parameter. Finds variables in each parameter and adds all of them to the list.
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
    
    #Scans each parameter. Finds constants in each parameter and adds all of them to the list.
    def findConstants(self):
        constants = []
        for i in self.params:
            constants = constants + i.findConstants()
        return constants
    
    #Scans each parameter. Finds repeats in each parameter and adds all of them to the list.
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        if self.isGround():
            repeats.append(self)
        return repeats
    
    #Is inherently a function. Finds functions in each parameter and adds all of them to the list.
    def findFunctions(self):
        functions = [self]
        for i in self.params:
            functions = functions + i.findFunctions()
        return functions
    
    #Returns a copy of the predicate with 'var' replaced with 'rpt'.
    #Should not need to copy, because it was done by the clause.
    def replace(self, var, rpt):
        #function = deepcopy(self)
        function = self
        for i in xrange(len(function.params)):
            term = function.params[i]
            if term==var:
                function.params[i] = rpt
            else:
                function.params[i] = term.replace(var, rpt)
        return function

print "Function structure loaded."

Function structure loaded.


In [6]:
#Implementation of predicate (very similar implementation to function). 
#Includes symbol (must be unique) and list of paramaters.
#Registry containts copies of all functions that may be altered in enumerate algorithm.
#Overrides str, len, eq, and hash
class Predicate:
    
    #Length calculated by definition: 1 + length of each parameter.
    def __init__(self, s, *args):
        self.sym = s
        self.params = list(args)
        self.length = 1+sum([len(i) for i in self.params])
       
    def __str__(self):
        toReturn = self.sym + "("
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + ")"
        return toReturn
    
    def __len__(self):
        return 1+sum([len(i) for i in self.params])
    
    #def __eq__(self, other):
    #    return isinstance(other, Predicate) and self.params == other.params
 
    def __eq__(self, other):
        return str(self) == str(other)
    
    def __hash__(self):
        return hash(str(self))
    
    #Previous naming scheme testing. Not in use.
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Function.registry])>0:
            return self.name(s + ".")
        else: return s
    
    #For each parameter (0 to arity), scans forward for repeated terms. Any repeats are replaced by Repeats.
    #Then reduces size of each parameter.
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()
            
    #Scans each parameter. If any are not ground, the entire instance of the function is not ground.
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True
    
    #Scans each parameter. Finds variables in each parameter and adds all of them to the list.
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
    
    #Scans each parameter. Finds constants in each parameter and adds all of them to the list.
    def findConstants(self):
        constants = []
        for i in self.params:
            constants = constants + i.findConstants()
        return constants
    
    #Scans each parameter. Finds repeats in each parameter and adds all of them to the list.
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        if self.isGround():
            repeats.append(self)
        return repeats
    
    #Scans each parameter. Finds functions in each parameter and adds all of them to the list.
    def findFunctions(self):
        functions = []
        for i in self.params:
            functions = functions + i.findFunctions()
        return functions

    #Returns a copy of the predicate with 'var' replaced with 'rpt'.
    #Should not need to copy, because it was done by the clause.
    def replace(self, var, rpt):
        #predicate = deepcopy(self)
        predicate = self
        for i in xrange(len(predicate.params)):
            term = predicate.params[i]
            if term==var:
                predicate.params[i] = rpt
            else:
                predicate.params[i] = term.replace(var, rpt)
        return predicate
    
print "Predicate structure loaded."

Predicate structure loaded.


In [7]:
#Implementation of clause (very similar implementation to function and predicate). 
#Includes list of paramaters.
#Registry containts copies of all functions that may be altered in enumerate algorithm.
#Overrides str, len, eq, and hash
class Clause: 
    
    #Length calculated by definition: sum of length of each parameter.
    def __init__(self, *args):
        self.params = list(args)
        self.length = sum([len(i) for i in self.params])
        
    def __str__(self):
        toReturn = "{"
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + "}"
        return toReturn
    
    def __len__(self):
        return sum([len(i) for i in self.params])
    
    #def __eq__(self, other):
    #    return isinstance(other, Clause) and self.params == other.params
    
    def __eq__(self, other):
        return str(self) == str(other)

    def __hash__(self):
        return hash(str(self))
    
    #For each parameter (0 to arity), scans forward for repeated terms. Any repeats are replaced by Repeats.
    #Then reduces size of each parameter. Returns new modified clause.
    def reduceSize(self):
        copy = deepcopy(self)
        for i in xrange(len(copy.params)):
            for j in xrange(i+1, len(copy.params)):
                if not isinstance(copy.params[j], Repeat) and copy.params[i] == copy.params[j]:
                    copy.params[j] = Repeat(copy.params[j])
        for i in copy.params:
            i.reduceSize()    
        return copy
    
    #Scans each parameter. If any are not ground, the entire instance of the function is not ground.
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True
    
    #Scans each parameter. Finds variables in each parameter and adds all of them to the list.
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
    
    #Scans each parameter. Finds constants in each parameter and adds all of them to the list.
    #Herbrand structures require at least one constant. If there are no constants, one arbitrary constant 'C' is added.
    def findConstants(self):
        constants = []
        for i in self.params:
            constants = constants + i.findConstants()
        if not constants:
            constants.append(Constant('C'))
        return constants
    
    #Scans each parameter. Finds repeats in each parameter and adds all of them to the list.
    #Unlike other structures, does not include itself.
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        return repeats
    
    #Scans each parameter. Finds functions in each parameter and adds all of them to the list.
    def findFunctions(self):
        functions = []
        for i in self.params:
            functions = functions + i.findFunctions()
        return functions
    
    #Returns a copy of the clause with 'var' replaced with 'rpt' to be used in deeper levels of recursion.
    def replace(self, var, rpt):
        clause = deepcopy(self)
        for i in xrange(len(clause.params)):
            term = clause.params[i]
            if term==var:
                clause.params[i] = rpt
            else:
                clause.params[i] = term.replace(var, rpt)
        return clause
    
print "Clause structure loaded."

Clause structure loaded.


<h2>The Algorithm</h2>
<p>Given a clause, enumerates all ground clauses whose size is less than some size bound.</p>

In [15]:
"""
Function:
    enumerateClauses
    
Use:
    Given a clause and sizebound, enumerates all ground clauses whose s_dag - calculated by len(reduceSize()) -
    is less than the desired sizebound These results can then be fed into a DPLL implementation to determine
    satisfiability. 

Inputs:
    clause (Clause): The desired clause for ground instances under a sizebound to be enumerated from.
    size (int): The sizebound. Conventionally compared to s_lin. Compared to s_dag here.
    rpts (set): The set of repeats. Is empty to start, used within algorithm. Do not pass anything here.
    clauseSet (set): The set of clauses that are both ground and under the sizebound. 
                     Is empty to start, used within algorithm. Do not pass anything here.
    
Output:
    clauseSet (set): The set of clauses that are both ground and under the sizebound after the terminating condition.
    
Helper Functions:
    hyperizefunctions: 
                        Returns a copy of a function with all terms replaced by new variables.
                        Inputs: 
                            functions (set): a set of functions.
                        Outputs:
                            hyfunctions (set): the same set of functions with all terms replaced with variables.
    
"""
       
def enumerateClauses(clause, size, rpts = set(), clauseSet = set()):
    
    #Reduced clause length. Calculation often, so we'll store it once.
    clauseReducedSize = len(clause.reduceSize())
    
    #Create an effective closure around repeats, (potentially) growing and shrinking with recursion depth.
    repeats = rpts.union(set(clause.findRepeats()))
    
    #Base Case: Clause size is larger than size bound. Does nothing to clauseSet and returns.
    if clauseReducedSize>size:
        return clauseSet
    
    #Base Case: The clause is ground. Add it to clauseSet and return.
    elif clause.isGround():
        clauseSet.add(clause)

    #Case 1: The clause has reached the sizebound, but is not ground. Recursively call enumerateClauses with only
    #terms of size 0 - repeats.
    elif clauseReducedSize==size:
        for x in clause.findVariables():
            for i in repeats:
                enumerateClauses(clause.replace(x,i), size, repeats, clauseSet)
                
    #Case 2: The clause has not reached the sizebound and is not ground. Recursively call enumerateClauses with
    #any possible goal-oriented term - constants, functions, or repeats.
    else:
        constants = set(clause.findConstants())
        variables = set(clause.findVariables())
        functions = hyperizefunctions(clause.findFunctions())
        for x in variables:
            for i in constants.union(repeats).union(functions):
                #Branch-and-bound approach to cut unnecessary enumerateClauses calls.
                #If the size of the term to be replaced is larger than the available space left, don't bother checking.
                if (size-clauseReducedSize-len(i))>=0:
                    enumerateClauses(clause.replace(x,i), size, repeats, clauseSet)

    return clauseSet
    
def hyperizefunctions(functions):
    hyfunctions = set()
    for fn in functions:
        hyperfn = deepcopy(fn)
        for i in xrange(len(hyperfn.params)):
            hyperfn.params[i] = Variable()
        hyfunctions.add(hyperfn)
    return hyfunctions

<h2>Testing Zone</h2>
<p>Instantiates some structures then runs the algorithm.</p>

In [16]:
a = Constant('a')
b = Constant('b')
c = Constant('c')
x = Variable('x')
y = Variable('y')
z = Variable('z')
f1 = Function('f', a, z, x, y, b)
f2 = Function('f', a, x)
g1 = Function('g', b, x)
g2 = Function('g', b, f1)
g3 = Function('g', c, f1)
p1 = Predicate('P',f1, f2, g2, g3)
p2 = Predicate('Q', f1)
p3 = Predicate('R', f1, g3)
c1 = Clause(p1, p2, p3)
c2 = Clause(f1)

for i in enumerateClauses(c2, 4):
    print i


{f(a,a,a,a,b)}
{f(a,b,a,a,b)}
{f(a,b,b,a,b)}
{f(a,a,a,b,b)}
{f(a,a,b,b,b)}
{f(a,b,b,b,b)}
{f(a,b,a,b,b)}
{f(a,a,b,a,b)}


Todo: 
    Comment everything, 
    Follow FOL statement through steps before DPLL, 
    Test on some problems (pelletier), 
    Look for efficiency improvements??